In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import random
import uuid
from math import sqrt

In [2]:
from matplotlib import rc
rc('animation', html='jshtml')

In [3]:
INIT_PREDATORS = 100        #Początkowa liczba drapieżników
INIT_PREYS = 50            #Początkowa liczba ofiar
X_MIN = 0
X_MAX = 2
Y_MIN = 0
Y_MAX = 2
global SAFE_DISTANCE
SAFE_DISTANCE = 0.5         #odległość potrzebna do ataku drapieżnika
global PREDATOR_EFFECTIVENESS
PREDATOR_EFFECTIVENESS = 30 #liczba z zakresu od 0-100 określająca szansę na śmierć ofiary w starciu z drapieżnikiem
global MAX_VITALITY
MAX_VITALITY = 50           #wytrzymałość drapieżników (ile rund jest w stanie wytrzymać bez jedzenia)
ITERATIONS = 500            #liczba iteracji / kroków w błądzeniu losowym, którą symulujemy
global STEP_SIZE
STEP_SIZE = 0.2

In [4]:
class SurfaceBoundaries:
    def __init__(self, x_min, x_max, y_min, y_max):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max

In [5]:
global surfaceBoundaries
surfaceBoundaries = SurfaceBoundaries(X_MIN, X_MAX, Y_MIN, Y_MAX)

In [6]:
class Position:
    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y
        
    def __str__(self):
        return ("Position(" + str(self.x) + ", " + str(self.y) + ")")
    
    def isWithinBoundaries(self) -> bool:
        global surfaceBoundaries
        return (
            surfaceBoundaries.x_min <= self.x <= surfaceBoundaries.x_max 
            and surfaceBoundaries.y_min <= self.y <= surfaceBoundaries.y_max
        )

    def adjust_to_boundaries(self) -> None:
        global surfaceBoundaries
        
        if not self.isWithinBoundaries():
            if self.x < surfaceBoundaries.x_min: self.x = surfaceBoundaries.x_min
            if self.x > surfaceBoundaries.x_max: self.x = surfaceBoundaries.x_max
            if self.y < surfaceBoundaries.y_min: self.y = surfaceBoundaries.y_min
            if self.y > surfaceBoundaries.y_max: self.y = surfaceBoundaries.y_max
                
    def random_step(self) -> None:
        phi = 2 * np.pi * random.uniform(0, 1)
        self.x += np.cos(phi) * STEP_SIZE
        self.y += np.sin(phi) * STEP_SIZE
        self.adjust_to_boundaries()

def randomPosition() -> Position:
    global surfaceBoundaries
    x = random.uniform(surfaceBoundaries.x_min, surfaceBoundaries.x_max)
    y = random.uniform(surfaceBoundaries.y_min, surfaceBoundaries.y_max)
    return Position(x, y)

def distance(position_a: Position, position_b: Position):
    return sqrt(
        pow(position_a.x - position_b.x, 2) + pow(position_a.y - position_b.y, 2)        
    )

In [7]:
def coin_flip(probability: float) -> bool:
    #accepts probability as number from 0 to 1
    #perform the binomial distribution (returns 0 or 1)    
    return np.random.binomial(1, probability)

def list_to_string(lst: list[object]) -> str:
    string_delimiter = ",\n"
    return string_delimiter.join(str(el) for el in lst)

In [8]:
class Predator:
    def __init__(self):
        global MAX_VITALITY

        self.id = uuid.uuid1()
        self.position = randomPosition()
        self.isAlive = True
        self.vitality = MAX_VITALITY

    def __str__(self):
        return ("Predator(" + 
            str(self.id) + ", " + 
            str(self.position) + ", " + 
            str(self.isAlive) + ", " + 
            str(self.vitality) + 
            ")")

In [9]:
global predators
predators = []
for i in range(INIT_PREDATORS):
    predators.append(Predator())

In [10]:
class Prey:
    def __init__(self):
        self.id = uuid.uuid1()
        self.position = randomPosition()
        self.isAlive = True
        
    def get_endangering_predators(self) -> list[Predator]:
        global predators
        global SAFE_DISTANCE
        
        def are_too_close(predator: Predator, prey: Prey) -> bool:
            return (distance(prey.position, predator.position) < SAFE_DISTANCE)

        return list(filter(lambda predator: are_too_close(self, predator), predators))
        
    def chance_to_die(self, endangering_predators_number: int) -> float:
        global PREDATOR_EFFECTIVENESS
        
        chance_to_survive = pow(((100 - PREDATOR_EFFECTIVENESS) / 100), endangering_predators_number)
        return 1 - chance_to_survive
    
    def __str__(self):
        return ("Prey(" + 
                str(self.id) + ", " + 
                str(self.position) + ", " + 
                str(self.isAlive) +
                ")")

In [11]:
global preys
preys = []
for i in range(INIT_PREYS):
    preys.append(Prey())

In [12]:
global dead_predators
dead_predators = []
global dead_preys
dead_preys = []

In [13]:
def find_endangered_preys_and_attacking_predators():
    global predators
    endangered_preys = list()
    for prey in preys:
        endangering_predators = prey.get_endangering_predators()
        
        if len(endangering_predators) > 0:
            chance_to_die = prey.chance_to_die(len(endangering_predators))
            endangered_preys.append((prey, endangering_predators, chance_to_die))
    return endangered_preys

In [14]:
def mark_predators_meal(happy_predators):
    global predators
    for predator in predators:
        if predator in happy_predators:
            predator.vitality = MAX_VITALITY

In [15]:
def clash_preys_and_predators(endangered_preys):
    for (prey, endangering_predators, chance_to_die) in endangered_preys:
        prey_dies = (coin_flip(chance_to_die) == 1) # == 1 converts 0/1 to False/True
        if prey_dies:
            print(str(prey) + " got killed by predators " + str(len(endangering_predators)))
            prey.isAlive = False
            mark_predators_meal(endangering_predators)


In [16]:
def decrement_predators_vitality():
    global predators
    for predator in predators:
        predator.vitality = predator.vitality - 1
        if predator.vitality <= 0:
            predator.isAlive = False

In [17]:
def filter_out_dead_predators():
    global predators
    global dead_predators
    
    new_dead_predators = list(filter(lambda predator: (predator.isAlive == False), predators))
    alive_predators = list(filter(lambda predator: (predator.isAlive == True), predators))
    predators = alive_predators
    dead_predators += new_dead_predators    

In [18]:
def filter_out_dead_preys():
    global preys
    global dead_preys
        
    new_killed_preys = list(filter(lambda prey: (prey.isAlive == False), preys))
    alive_preys = list(filter(lambda prey: (prey.isAlive == True), preys))
    preys = alive_preys
    dead_preys += new_killed_preys

In [19]:
def move_alive_animals_by_one_random_step():
    global predators
    global preys
    
    for predator in predators:
        predator.position.random_step()
    for prey in preys:
        prey.position.random_step()    

In [20]:
for i in range(ITERATIONS):
    
    endangered_preys = find_endangered_preys_and_attacking_predators()
    clash_preys_and_predators(endangered_preys)
    decrement_predators_vitality()
    filter_out_dead_predators()
    filter_out_dead_preys()
    move_alive_animals_by_one_random_step()
    
    print("Iteration " + str(i) + ":")
    print("Dead preys: " + str(len(dead_preys)))
    print("Dead predators: " + str(len(dead_predators)))
    print("Alive preys: " + str(len(preys)))
    print("Alive predators: " + str(len(predators)))
    print("\n")  

Prey(9e20d4c4-da1b-11ec-8ea0-acde48001122, Position(1.3772026128011519, 0.5816747518130758), True) got killed by predators Predator(9e1ee4b6-da1b-11ec-8ea0-acde48001122, Position(1.2961653651732201, 0.2943086257095773), True, 50),
Predator(9e1ee510-da1b-11ec-8ea0-acde48001122, Position(1.341521145098026, 0.2818616219978334), True, 50),
Predator(9e1ee57e-da1b-11ec-8ea0-acde48001122, Position(1.4002912942733952, 0.37202512197903403), True, 50),
Predator(9e1ee628-da1b-11ec-8ea0-acde48001122, Position(1.541679785482386, 1.0174940595775963), True, 50),
Predator(9e1ee6f0-da1b-11ec-8ea0-acde48001122, Position(1.6339743987485544, 0.7592910740529792), True, 50),
Predator(9e1ee812-da1b-11ec-8ea0-acde48001122, Position(1.038761833494042, 0.6247992298142642), True, 50),
Predator(9e1ee86c-da1b-11ec-8ea0-acde48001122, Position(0.9742930147062889, 0.3137390302757972), True, 50),
Predator(9e1ee97a-da1b-11ec-8ea0-acde48001122, Position(1.3125097396079435, 0.30844277662967023), True, 50),
Predator(9e1ee

Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 305:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 306:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 307:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 308:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 309:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 310:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 311:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 312:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 313:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 314:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Alive predators: 0


Iteration 315:
Dead preys: 50
Dead predators: 100
Alive preys: 0
Aliv